In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# Concaténer plusieurs fichiers Excel

Ce notebook va :

1. Parcourir un dossier contenant plusieurs fichiers `.xlsx` (même structure).
2. Lire pour chaque fichier les deux feuilles : `General Info` et `Data`.
3. Extraire Latitude / Longitude de `General Info` (plusieurs formats possibles gérés automatiquement).
4. Ajouter ces valeurs (et le nom du fichier source) à chaque ligne de la feuille `Data`.
5. Concaténer toutes les feuilles `Data` en un seul DataFrame.
6. Sauvegarder le résultat en `combined.xlsx` et `combined.csv` dans le même dossier.

Modifiez simplement la variable `BASE_DIR` ci‑dessous pour pointer vers votre dossier de fichiers Excel.


In [4]:
# Paramètres utilisateur
# Chemin du dossier contenant les fichiers Excel
BASE_DIR = "/Users/galex/Desktop/StageNUS/Refs/Mangroves/Mangrove Datasets/All Datasets" 
# Motif pour sélectionner les fichiers (tous les .xlsx hors ~ temporaires)
FILE_GLOB = "*.xlsx"
OUTPUT_BASENAME = "combined"

# Fonctions utilitaires

def read_general_info_latlon(df_gi: pd.DataFrame):
    """Tente d'extraire latitude / longitude depuis la feuille General Info.
    Cherche des colonnes ou clés ressemblant à Latitude / Longitude (case-insensitive).
    Gère les formats où la feuille est soit:
      - un tableau clé/valeur (2 colonnes) 
      - un tableau déjà avec colonnes nommées
    Retourne (lat, lon) ou (np.nan, np.nan) si absent.
    """
    if df_gi.empty:
        return np.nan, np.nan
    # Normaliser noms de colonnes
    cols_lower = [c.strip().lower() for c in df_gi.columns]

    lat, lon = np.nan, np.nan

    def extract_from_series(s):
        nonlocal lat, lon
        key = str(s[0]).strip().lower()
        if 'lat' in key and pd.isna(lat):
            try:
                lat = float(str(s[1]).replace(',', '.'))
            except Exception:
                pass
        if 'lon' in key or 'long' in key:
            if pd.isna(lon):
                try:
                    lon = float(str(s[1]).replace(',', '.'))
                except Exception:
                    pass

    # Cas 1: type clé/valeur (au moins 2 colonnes et la première contient des clés)
    if len(df_gi.columns) >= 2:
        # Vérifier si première colonne semble contenir des clés textuelles
        first_col_text_ratio = (
            df_gi.iloc[:,0].apply(lambda x: isinstance(x, str)).mean()
            if len(df_gi) else 0
        )
        if first_col_text_ratio > 0.5:
            for _, row in df_gi.iloc[:, :2].iterrows():
                extract_from_series(row)
    
    # Cas 2: colonnes nommées contiennent directement lat/lon
    for c in df_gi.columns:
        cl = c.strip().lower()
        if 'lat' in cl and pd.isna(lat):
            try:
                lat = float(str(df_gi[c].dropna().iloc[0]).replace(',', '.'))
            except Exception:
                pass
        if ('lon' in cl or 'long' in cl) and pd.isna(lon):
            try:
                lon = float(str(df_gi[c].dropna().iloc[0]).replace(',', '.'))
            except Exception:
                pass

    return lat, lon


def process_file(path: Path):
    try:
        xls = pd.ExcelFile(path)
    except Exception as e:
        print(f"[WARN] Impossible d'ouvrir {path.name}: {e}")
        return None

    missing = []
    sheet_gi = 'General info'
    sheet_data = 'Data'

    if sheet_gi not in xls.sheet_names:
        missing.append(sheet_gi)
    if sheet_data not in xls.sheet_names:
        missing.append(sheet_data)
    if missing:
        print(f"[WARN] Feuilles manquantes {missing} dans {path.name}")
        return None

    try:
        df_gi = pd.read_excel(xls, sheet_gi)
    except Exception as e:
        print(f"[WARN] Lecture General Info échouée pour {path.name}: {e}")
        df_gi = pd.DataFrame()
    try:
        df_data = pd.read_excel(xls, sheet_data)
    except Exception as e:
        print(f"[WARN] Lecture Data échouée pour {path.name}: {e}")
        return None

    lat, lon = read_general_info_latlon(df_gi)

    df_data = df_data.copy()
    df_data['SourceFile'] = path.name
    df_data['Latitude'] = lat
    df_data['Longitude'] = lon
    return df_data

# Exécution
base = Path(BASE_DIR)
if not base.exists():
    raise FileNotFoundError(f"Le dossier spécifié n'existe pas: {base}")

files = [p for p in base.glob(FILE_GLOB) if p.is_file() and not p.name.startswith('~$')]
if not files:
    raise RuntimeError(f"Aucun fichier Excel trouvé dans {base} avec le motif {FILE_GLOB}")

parts = []
for f in files:
    print(f"Traitement: {f.name}")
    df_part = process_file(f)
    if df_part is not None:
        parts.append(df_part)

if not parts:
    raise RuntimeError("Aucun fichier valide n'a pu être traité.")

combined = pd.concat(parts, ignore_index=True)
print(f"Lignes totales concaténées: {len(combined)}")

# Sauvegarde
out_xlsx = base / f"{OUTPUT_BASENAME}.xlsx"
out_csv = base / f"{OUTPUT_BASENAME}.csv"

try:
    combined.to_excel(out_xlsx, index=False)
    print(f"Fichier Excel sauvegardé: {out_xlsx}")
except Exception as e:
    print(f"[WARN] Echec sauvegarde Excel: {e}")

try:
    combined.to_csv(out_csv, index=False)
    print(f"Fichier CSV sauvegardé: {out_csv}")
except Exception as e:
    print(f"[WARN] Echec sauvegarde CSV: {e}")

# Aperçu
combined.head()

Traitement: SWAMP Data-Soil carbon-Acarau Boca-2016-Brazil.xlsx
Traitement: SWAMP Data-Soil carbon-Bunaken-2011.xlsx
Traitement: SWAMP Data-Soil carbon-Ca Mau-2012-Vietnam.xlsx
Traitement: SWAMP Data-Soil carbon-Can Gio-2012-Vietnam.xlsx
Traitement: SWAMP Data-Soil carbon-Case Shell-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Jardin Du Elephant-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Kubu Raya-2011-Indonesia.xlsx
Traitement: SWAMP Data-Soil carbon-Lac Simba Deux-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Lac Simba-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Lac Sounga Deux-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Lac Sounga-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Mwana Mouele South-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Mwana Mouele-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Ndougou-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Paga-2014.xlsx
Traitement: SWAMP Data-Soil carbon-Sembilang-2011-Indonesia.xlsx
Traitement: SWAMP Data-Soil carbon-Teminabuan-2011-Indo

,No ID,Site ID,Plot,Sub-plot,Depth interval (cm),Collected sample depth/depth range (cm),Soil organic matter depth (cm),Bag. No,Wet weight (gr),Dry weight (g),...,BGC (MgC/ha),Basal area per ha (m2),AGB summed per plot (Mg/ha),BGB summed per plot (Mg/ha),AGC summed per plot (MgC/ha),BGC summed per plot (MgC/ha),Basal area (m2/ha) summed per plot,source for density,source for AGB allometry,source for BGB allometry
0,1,ACA,Acarau Boca,1,0-15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ACA,Acarau Boca,1,15-30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,ACA,Acarau Boca,1,30-50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,ACA,Acarau Boca,1,50-100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,ACA,Acarau Boca,1,>100,NaN,210.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
